This notebook uses the indexes of the 2000 most popular games supplied by our preliminary cleaning notebook. We will scrape 2000 reviews from each of these games using the boardgamegeek api.

In [1]:
import pandas as pd
import numpy as np
import requests
import time
import datetime
from bs4 import BeautifulSoup

In [2]:
to_scrape = pd.read_csv('cleaned_data/high_review_count_games')
to_scrape.columns = ['game_id', 'num_ratings']
to_scrape.head()

,game_id,num_ratings
0,822,98777
1,13,98557
2,68448,82043
3,36218,76396
4,9209,69577


Each request we make will be for 100 reviews(maximum per page) for each of 100 games (we could do more, but we run into issues as our index numbers get bigger). We will write into a csv the game id, user_id, rating (out of 10), and comment if available for each rating. We will limit the length of our csvs to 100,000 items, so we will be looking for around 40 csv files to be created. We will use these files in another notebook to begin the process of creating our recommendation system.

In [3]:
start = datetime.datetime.now()

pages = range(1, 21)
comment_counter = 1
csv_counter = 1
with open(f'scraped_ratings/ratings{csv_counter}.csv','w', newline='') as file:
    while len(to_scrape) > 0:
        this_batch = to_scrape[:100]
        string_ids = ''
        for game in this_batch.game_id:
            string_ids += (str(game) + ',')
        for page in pages:
            response = requests.get(f'https://www.boardgamegeek.com/xmlapi2/thing?id={string_ids}&ratingcomments=1&pagesize=100&page={page}')
            if response.status_code == 404:
                print('404', page)
                
            elif response.status_code == 200:
                    content = response.content
                    soup = BeautifulSoup(content, "xml")
                    items = soup.findAll('item')
                    for item in items:
                        game_id = item.get('id')
                        comments = item.findAll('comment')
                        for comment in comments:
                            file.write(f"{game_id}\t{comment.get('username')}\t{comment.get('rating')}\t{comment.get('value')}")
                            file.write('\n')
                            comment_counter += 1
                            if comment_counter > 100000:
                                csv_counter += 1
                                comment_counter = 1
                                file = open(f'scraped_ratings/ratings{csv_counter}.csv','w', newline='')

            else:
                print(response.status_code)
                print(csv_counter, page)
                time.sleep(15)
        
            if comment_counter > 100000:
                csv_counter += 1
                comment_counter = 1
                file = open(f'scraped_ratings/ratings{csv_counter}.csv','w', newline='')
                                       
        to_scrape.drop(this_batch.index, axis=0, inplace=True)

        time.sleep(3)
        print('Set Complete')
end= datetime.datetime.now()
print(end-start)
len(to_scrape)

Set Complete
Set Complete
Set Complete
Set Complete
Set Complete
Set Complete
Set Complete
Set Complete
429
17 4
429
18 15
Set Complete
429
19 8
Set Complete
429
21 4
429
22 15
Set Complete
429
23 7
429
23 16
Set Complete
429
25 13
Set Complete
429
26 8
429
27 18
Set Complete
429
29 15
Set Complete
429
30 11
Set Complete
429
32 5
429
33 16
Set Complete
429
34 7
429
35 18
Set Complete
429
36 11
429
36 20
Set Complete
429
37 10
Set Complete
0:16:32.009875


0